First of all we are going to reserve this first chunk of code to import all the libraries that we will be using along this assignment

In [324]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time 
import random
from numpy.linalg import inv
from scipy.optimize import minimize

Once we have the libraries, we are going to generate a random data sample for a regression model, considering the contraints proposed in the problem:

 - 0 $\leq \beta_{i} \leq 5 \hspace{0.5cm}$ i = 0,...,K for K at least 1000 independent variables and n=5000 observations

In [79]:
random.seed(10)

K=1000

n_train=50000
n_test=2000
n=n_train+n_test
nvars=1000

randombeta = np.random.randint(0,6,size=([nvars+1,1]))
randomerror = np.random.normal(0,1,(n,1))

X0 = np.ones([n,1]) # the first column has all values equal to one for the coefficients of beta_0
X1 = np.random.uniform(0,10,([n,nvars]))
randomX = np.concatenate([X0, X1],axis=1)

y = np.dot(randomX,randombeta) + randomerror

X = randomX[0:n_train,:]
Y = y[0:n_train]

X_test = randomX[(n_train+1):n,:]
Y_test = y[(n_train+1):n]



## a) (0.5 points) Estimate the value of the regression coefficients by implementing the analytical
solution. Use this solution as a benchmark for the following sections.

We have to minimize:

\begin{align*}
  \text{minimize}_\beta \quad & \|y-X\beta\|^2_2 + \rho\|\beta\|^2_2
\end{align*}

We can rewrite the formula as:

\begin{align*}
  (y-X\beta)^T(y-X\beta) + \rho(\beta^T\beta)
\end{align*}

\begin{align*}
  Y^TY - Y^TX\beta - \beta^TX^TY + \beta^TX^T\beta X + \rho (\beta^T \beta)
\end{align*}

Thus, for minimizing we have to take the partial derivative and equal it to 0.

\begin{align*}
  \frac{\partial F}{\partial\beta} = 0  
\end{align*}

\begin{align*}
  \frac{\partial F}{\partial\beta} = -2X^TY + 2X^TX\beta +  2\rho \beta  
\end{align*}

\begin{align*}
  -2X^TY + 2X^TX\beta + 2\rho \beta = 0
\end{align*}

Now taking the variable $\beta$ to one of the sides of the equality from the first derivative we obtain the analytical solution of the preceding problem, which is: 

\begin{align*}
  \beta_{ls}=(X^T X + pI)^{-1}X^T Y
\end{align*}

We can also calculate the second derivative which corresponds to the hessian as this will be also used in the following points of this project.

\begin{align*}
  \frac{\partial^2 F}{\partial\beta} =  2X^TX +  2\rho  
\end{align*}
 


We can now implement the equation given by the derivative in python and obtain what would be the exact solution of the prolem. This solution will be use as a beenchmark for the results that we will be obtaining later on other optimization model.

In [326]:
time_start_exact = time.time()

beta_ls_exact = np.dot(np.dot(inv(np.dot((X.T),X)+(np.identity(np.dot(X.T,X).shape[0]))),X.T),Y)

time_elapsed_exact = (time.time() - time_start_exact)

print('Values of the (exact) least squares coefficients:')
for i in range(nvars+1):
    print('beta %3d %7.3f' %(i,beta_ls_exact[i]))
print('Elapsed time = %8.5f' %(time_elapsed_exact))


Values of the (exact) least squares coefficients:
beta   0   2.747
beta   1   2.001
beta   2   4.000
beta   3   4.998
beta   4   1.999
beta   5   1.000
beta   6   2.001
beta   7   3.001
beta   8   4.001
beta   9   1.002
beta  10   4.998
beta  11  -0.001
beta  12   1.000
beta  13   3.998
beta  14   5.001
beta  15   1.000
beta  16   0.000
beta  17   0.999
beta  18   0.997
beta  19   2.997
beta  20   2.002
beta  21   2.001
beta  22   4.002
beta  23   5.001
beta  24   0.999
beta  25   3.997
beta  26  -0.001
beta  27   3.002
beta  28   4.000
beta  29   0.001
beta  30   1.998
beta  31   4.002
beta  32   2.999
beta  33   5.000
beta  34   4.996
beta  35   5.004
beta  36   4.000
beta  37   3.002
beta  38   3.000
beta  39   1.999
beta  40   2.002
beta  41   4.000
beta  42   4.996
beta  43   0.002
beta  44   1.000
beta  45   4.001
beta  46   2.998
beta  47   1.001
beta  48   2.001
beta  49   4.998
beta  50   0.999
beta  51   2.002
beta  52   2.000
beta  53   2.998
beta  54   1.003
beta  55   3.00

We can see that at prior the betas obtained are reasonable with the betas generated for the dataset. Indeed we can see that almost none of them are above 5 and under 0, which were the constrains impossed while generating the dataset for the betas.

Also, we can see that the computation time is very low, it only took over 4 seconds to obtain what would be the exact solution.

In [327]:
print("Max: ",max(beta_ls_exact),"Min: ",min(beta_ls_exact))

Max:  [5.0050469] Min:  [-0.00359519]


Within this output we can clearly see that the boundaries impossed are fulfilled almost perfectly.


## b) (1 points) Estimate the value of the regression coefficients by using the function minimize from the Python module Scipy.optimize. Try at least four available solvers and compare their performance in terms of number of iterations, number of function, gradient and hessian evaluations as well as total computational time.

To do this we are going to define different equations from our objective function that will be neccesary to implement different algorithms and to compare the time taken and the results obtained.

Therefore we will implement:

- The objective function defined in the main problem. 
- The first derivative correspondant to the gradient
- The second derivative or the hessian

All these derivations have been calculated above, so we will now implement them in python as functions.

In [328]:

def ridreg(beta_ls, X, Y,p):
    beta_ls = np.matrix(beta_ls)
    z = Y - np.dot(X,beta_ls.T)
    a=np.dot(z.T,z)
    b=np.dot(beta_ls,beta_ls.T)
    return (a+p*b)

def ridreg_der(beta_ls,X,Y,p):
    beta_ls = np.matrix(beta_ls)
    pp = -2*np.dot((Y-np.dot(X,(beta_ls).T)).T,X) + 2*p*beta_ls
    aa = np.squeeze(np.asarray(pp))
    return aa

def ridreg_hess(beta_ls,X,Y,p):
    ss = 2*np.dot(np.transpose(X),X) + 2*p*np.identity(nvars+1)
    return ss


We will now try to estimate the regression coefficient by different algorithms available in the scipy-optimze module in Python. First of all we are going to implement an algorithm using a method called "dogleg", which takes all the functions defined, the objective funciton, the derivative and the hessian.

In [336]:
beta_ls0 = np.zeros(nvars+1)

time_start_dogleg = time.time()

res = minimize(ridreg, beta_ls0, args=(X, Y,5), method='dogleg',jac=ridreg_der,hess=ridreg_hess, options={'disp': True,'xtol': 1e-4})

time_elapsed_dogleg = (time.time() - time_start_dogleg)

print('\nValues of the least squares coefficients obtained with dogleg:')
for i in range(nvars+1):
    print('beta %3d %7.3f' %(i,res.x[i]))

err_val_0 = np.linalg.norm(beta_ls_exact.T-res.x,ord=2)/np.linalg.norm(beta_ls_exact.T,ord=2)
print('\nError in values of coefficients = %8.4f' %err_val_0)

print('Elapsed time = %8.5f' %(time_elapsed_dogleg))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/optimize/_minimize.py:697: OptimizeWarning: Unknown solver options: xtol
  res = _minimize_dogleg(fun, x0, args, jac, hess,


         Current function value: 96469.832652
         Iterations: 7
         Function evaluations: 9
         Gradient evaluations: 8
         Hessian evaluations: 8

Values of the least squares coefficients obtained with dogleg:
beta   0   2.328
beta   1   2.001
beta   2   4.000
beta   3   4.998
beta   4   1.999
beta   5   1.000
beta   6   2.001
beta   7   3.001
beta   8   4.001
beta   9   1.002
beta  10   4.998
beta  11  -0.001
beta  12   1.000
beta  13   3.998
beta  14   5.001
beta  15   1.000
beta  16   0.000
beta  17   0.999
beta  18   0.997
beta  19   2.997
beta  20   2.002
beta  21   2.001
beta  22   4.002
beta  23   5.002
beta  24   0.999
beta  25   3.997
beta  26  -0.001
beta  27   3.002
beta  28   4.000
beta  29   0.001
beta  30   1.998
beta  31   4.002
beta  32   2.999
beta  33   5.000
beta  34   4.996
beta  35   5.004
beta  36   4.001
beta  37   3.002
beta  38   3.000
beta  39   1.999
beta  40   2.002
beta  41   4.000
beta  42   4.996
beta  43   0.002
beta  44   1.000
beta

We can see that the results obtained in within this methods are pretty good. First of all we can check the numebr of iterations, only 7. Then we can see the number of evaluations, 9 for the objective function and 8 for the derivative and the hessian, which are indeed pretty low. These results will come with a low computation time, which indeed, it only takes 8 seconds to run and the results obtained are surprisingly good, almost no error at all.

We are now going to use the Newtwon-CG method, which uses again both evaluations, Hessian and Gradient.

In [337]:
beta_ls1=np.zeros(nvars+1)

time_start_Newton = time.time()

res = minimize(ridreg, beta_ls1, args=(X, Y,5), method='Newton-CG', jac=ridreg_der, hess=ridreg_hess, options={'disp': True,'xtol': 1e-4})

time_elapsed_Newton = (time.time() - time_start_Newton)

print('\nValues of the least squares coefficients obtained with Newton-CG:')
for i in range(nvars+1):
    print('beta %3d %7.3f' %(i,res.x[i]))

err_val_2 = np.linalg.norm(beta_ls_exact.T-res.x,ord=2)/np.linalg.norm(beta_ls_exact.T,ord=2)
print('\nError in values of coefficients = %8.4f' %err_val_2)

print('Elapsed time = %8.5f' %(time_elapsed_Newton))

Optimization terminated successfully.
         Current function value: 96540.383885
         Iterations: 8
         Function evaluations: 9
         Gradient evaluations: 9
         Hessian evaluations: 8

Values of the least squares coefficients obtained with Newton-CG:
beta   0   0.515
beta   1   2.001
beta   2   4.001
beta   3   4.998
beta   4   1.999
beta   5   1.001
beta   6   2.001
beta   7   3.001
beta   8   4.001
beta   9   1.002
beta  10   4.998
beta  11  -0.001
beta  12   1.001
beta  13   3.998
beta  14   5.002
beta  15   1.000
beta  16   0.001
beta  17   1.000
beta  18   0.997
beta  19   2.998
beta  20   2.002
beta  21   2.001
beta  22   4.002
beta  23   5.002
beta  24   1.000
beta  25   3.998
beta  26  -0.000
beta  27   3.002
beta  28   4.000
beta  29   0.001
beta  30   1.998
beta  31   4.002
beta  32   2.999
beta  33   5.001
beta  34   4.997
beta  35   5.004
beta  36   4.001
beta  37   3.002
beta  38   3.001
beta  39   1.999
beta  40   2.002
beta  41   4.000
beta  42   4.9

From this method we have also obtained pretty good results. We can see that it has only 8 iteractions, 9 evaluations of the function and gradient and just 8 with the hessian, which are just a few more than the ones done for the dogleg method. However, in this case we can see that the error obtained is greater than before. Nevertheless it is still a pretty low error and the solution obtained is still good. However, the time taken is lower than the previous method. Overall we could say that this method works also fine but a little bit worse than dogleg.

We are now going to try with the BFGS method, which does not use the Hessian information.

In [338]:
beta_ls2=np.zeros(nvars+1)

time_start_BFGS = time.time()

res = minimize(ridreg, beta_ls2, args=(X, Y,5), method='BFGS', jac=ridreg_der, options={'disp': True,'xtol': 1e-4})

time_elapsed_BFGS = (time.time() - time_start_BFGS)

print('\nValues of the least squares coefficients obtained with BFGS:')
for i in range(nvars+1):
    print('beta %3d %7.3f' %(i,res.x[i]))

err_val_3 = np.linalg.norm(beta_ls_exact.T-res.x,ord=2)/np.linalg.norm(beta_ls_exact.T,ord=2)
print('\nError in values of coefficients = %8.4f' %err_val_3)

print("Time taken for the BFGS method:", time_elapsed_BFGS)


/var/folders/k5/qpnm3ydn32l0mc6qzyk6129c0000gn/T/ipykernel_10911/1618234040.py:5: OptimizeWarning: Unknown solver options: xtol
  res = minimize(ridreg, beta_ls2, args=(X, Y,5), method='BFGS', jac=ridreg_der, options={'disp': True,'xtol': 1e-4})


         Current function value: 96469.832755
         Iterations: 434
         Function evaluations: 983
         Gradient evaluations: 971

Values of the least squares coefficients obtained with Nelder-Mead:
beta   0   2.328
beta   1   2.001
beta   2   4.000
beta   3   4.998
beta   4   1.999
beta   5   1.000
beta   6   2.001
beta   7   3.001
beta   8   4.001
beta   9   1.002
beta  10   4.998
beta  11  -0.001
beta  12   1.000
beta  13   3.998
beta  14   5.001
beta  15   1.000
beta  16   0.000
beta  17   0.999
beta  18   0.997
beta  19   2.997
beta  20   2.002
beta  21   2.001
beta  22   4.002
beta  23   5.002
beta  24   0.999
beta  25   3.997
beta  26  -0.001
beta  27   3.002
beta  28   4.000
beta  29   0.001
beta  30   1.998
beta  31   4.002
beta  32   2.999
beta  33   5.000
beta  34   4.996
beta  35   5.004
beta  36   4.001
beta  37   3.002
beta  38   3.000
beta  39   1.999
beta  40   2.002
beta  41   4.000
beta  42   4.996
beta  43   0.002
beta  44   1.000
beta  45   4.001
beta  46

We can see that this case was way less efficient than the one obtained before. This method has done 434 iterations and has evaluated both function and gradient almost 990 times each. However, the results obtained are also very good, in fact they are apparently as good as the ones obtained in the firs case if we only consider the error. However, this method took much more to compute more than the "Newton-CG". So we could conclude that the hessian helps to obtain a solution in a much faster way but it does not neccesarily give better results.

Finally we will be using "trust-constr", which does not use the gradient.

In [342]:
beta_ls4=np.zeros(nvars+1)
time_start_TC = time.time()

res = minimize(ridreg, beta_ls4, args=(X, Y,5), method='trust-constr', hess=ridreg_hess, options={'disp': True,'xtol': 1e-4})
time_elapsed_TC = (time.time() - time_start_TC)

print('\nValues of the least squares coefficients obtained with Trust-Constr:')
for i in range(nvars+1):
    print('beta %3d %7.3f' %(i,res.x[i]))

err_val_4 = np.linalg.norm(beta_ls_exact.T-res.x,ord=2)/np.linalg.norm(beta_ls_exact.T,ord=2)
print('\nError in values of coefficients = %8.4f' %err_val_4)

print("Time taken for the BFGS method:", time_elapsed_TC)

`xtol` termination condition is satisfied.
Number of iterations: 15, function evaluations: 8017, CG iterations: 50, optimality: 1.31e-01, constraint violation: 0.00e+00, execution time: 1.8e+02 s.

Values of the least squares coefficients obtained with Trust-Constr:
beta   0   2.329
beta   1   2.001
beta   2   4.000
beta   3   4.998
beta   4   1.999
beta   5   1.000
beta   6   2.001
beta   7   3.001
beta   8   4.001
beta   9   1.002
beta  10   4.998
beta  11  -0.001
beta  12   1.000
beta  13   3.998
beta  14   5.001
beta  15   1.000
beta  16   0.000
beta  17   0.999
beta  18   0.997
beta  19   2.997
beta  20   2.002
beta  21   2.001
beta  22   4.002
beta  23   5.002
beta  24   0.999
beta  25   3.997
beta  26  -0.001
beta  27   3.002
beta  28   4.000
beta  29   0.001
beta  30   1.998
beta  31   4.002
beta  32   2.999
beta  33   5.000
beta  34   4.996
beta  35   5.004
beta  36   4.001
beta  37   3.002
beta  38   3.000
beta  39   1.999
beta  40   2.002
beta  41   4.000
beta  42   4.996
be

For this last case without using the derivative we can see that the time taken is larger than all the previous methods, which all used the derivative. Nevertheless, the number of interactions is not that high compare to the last one, but the function evaluation is also very high.

The results obtained at the end are as good as some of the other ones obtained so far. So again, using the hessian without the gradient also gives good results, but having boths helps to compute it faster.



## c) (1 points) Modify the preceding optimization model by adding (lower and upper) bounds on the values of the β coefficients. Solve it again with the module Scipy.optimize a by using (at least) two different solvers, which should accept the introduction of bounds on the variables. Compare these methods and briefly comment on possible interpretations of the values of the coefficients.

Following the same methodology that we have been using we are going to try now different optimizations methods from scipy that allows to include boundaries on the betas selected for minimizing our function. At prior this should help to converge faster and obtain better solutions, as we know for a fact that none of the betas should be over 5 and lower 0.

First of all we will be using the TNC method, which does not use the hessian information. Therefore, we are expecting it to still take a little bit of time more than the methods that used both information but yet less than the BFGS method which also used only the derivative.

In [340]:
beta_ls5=np.zeros(nvars+1)
time_start_TNC_bounds = time.time()

res = minimize(ridreg, beta_ls5, args=(X, Y,5), method='TNC', bounds=(((0,5),)*len(beta_ls5)), jac=ridreg_der, options={'disp': True,'xtol': 1e-4})

time_elapsed_TNC_bounds = (time.time() - time_start_TNC_bounds)

print('\nValues of the least squares coefficients obtained with TNC:')
for i in range(nvars+1):
    print('beta %3d %7.3f' %(i,res.x[i]))

err_val_5 = np.linalg.norm(beta_ls_exact.T-res.x,ord=2)/np.linalg.norm(beta_ls_exact.T,ord=2)

print('\nError in values of coefficients = %8.4f' %err_val_5)

print('Elapsed time = %8.5f' %(time_elapsed_TNC_bounds))

print("Max: ",max(res.x),"Min: ",min(res.x))

  NIT   NF   F                       GTG
    0    1  8.270298133797866E+12   4.13634595E+22
tnc: fscale = 9.8338e-13
tnc: stepmx = 1000
    1    4  1.227644610251825E+12   6.13497159E+21
    2    7  8.096201718909660E+10   4.04218523E+20
    3   10  7.271389756781909E+07   1.09938447E+14
tnc: fscale = 1.90746e-08
    4   13  1.193394877527927E+07   1.72836688E+13
    5   17  1.207367374980422E+05   3.57937204E+10
tnc: fscale = 1.05713e-06
    6   21  1.173257088935548E+05   3.07273557E+10
    7   25  1.151075540685302E+05   2.74355858E+10
    8   29  1.074092212158234E+05   1.61085607E+10
    9   33  1.033630539374590E+05   1.00926700E+10
   10   37  1.024561418559850E+05   8.72796290E+09
   11   41  1.024146578068262E+05   8.66068161E+09
   12   45  1.011621551222641E+05   6.81690080E+09
   13   49  1.007688432653747E+05   6.24088486E+09
   14   53  1.007218422980507E+05   6.16949163E+09
   15   57  1.000555256079893E+05   5.18520593E+09
   16   61  9.997549462599496E+04   5.06736939E


Values of the least squares coefficients obtained with TNC:
beta   0   2.413
beta   1   2.001
beta   2   4.001
beta   3   4.998
beta   4   1.999
beta   5   1.000
beta   6   2.001
beta   7   3.001
beta   8   4.001
beta   9   1.002
beta  10   4.998
beta  11   0.000
beta  12   1.000
beta  13   3.998
beta  14   5.000
beta  15   1.000
beta  16   0.000
beta  17   0.999
beta  18   0.997
beta  19   2.998
beta  20   2.002
beta  21   2.001
beta  22   4.002
beta  23   5.000
beta  24   0.999
beta  25   3.997
beta  26   0.000
beta  27   3.002
beta  28   4.000
beta  29   0.001
beta  30   1.998
beta  31   4.002
beta  32   2.999
beta  33   5.000
beta  34   4.997
beta  35   5.000
beta  36   4.001
beta  37   3.002
beta  38   3.000
beta  39   1.999
beta  40   2.002
beta  41   4.000
beta  42   4.996
beta  43   0.002
beta  44   1.001
beta  45   4.001
beta  46   2.998
beta  47   1.001
beta  48   2.001
beta  49   4.998
beta  50   0.999
beta  51   2.002
beta  52   2.000
beta  53   2.998
beta  54   1.003
beta

tnc: |xn-xn-1] = 1.71433e-05 -> convergence
  159  697  9.661165688300032E+04   7.42797239E+04
tnc: Converged (|x_n-x_(n-1)| ~= 0)


Imposing the bandwith should make it easier to find a solution and indeed a better one. We can see that the constraints impossed in the betas are being fulfilled, as there is no single beta over 5 and under 0. Also, we can see that we are obtaining a very low error for the coefficients, which indicates that the betas are very well calculated.

Also, we can see that impossing bandwiths also make it easier to compute. This method only took about 57 seconds, whereas the BFGS took two minutes.

We are going to try now with another method that uses both hessian and derivative to see if we can obtain even better results.

In [341]:
beta_ls6=np.zeros(nvars+1)
time_start_tk = time.time()

res = minimize(ridreg, beta_ls6, args=(X, Y,5), method='trust-krylov', bounds=(((0,5),)*len(beta_ls6)), jac=ridreg_der, hess=ridreg_hess, options={'disp': True})
time_elapsed_tk = (time.time() - time_start_tk)

## Print results
print('\nValues of the least squares coefficients obtained with Trust-Krylov:')
for i in range(nvars+1):
    print('beta %3d %7.3f' %(i,res.x[i]))

err_val_6 = np.linalg.norm(beta_ls_exact.T-res.x,ord=2)/np.linalg.norm(beta_ls_exact.T,ord=2)
print('\nError in values of coefficients = %8.4f' %err_val_6)

print('Elapsed time = ',(time_elapsed_tk))

 TR Solving trust region problem, radius 1.000000e+00; starting on first irreducible block
 TR Coldstart. Seeking suitable initial Î»â, starting with 0
 TR Starting Newton iteration for Î»â with initial choice 0.000000e+00
 TR  iter        Î»            dÎ»       âhâ(Î»)â-radius
 TR      1  2.008790e+11  2.008790e+11 -2.220446e-16


 iter inewton type    objective     Î³áµ¢ââ|háµ¢|      leftmost         Î»             Î³             Î´             Î±             Î²       

     0     1  cg_b -2.021295e+11  5.491307e+05  0.000000e+00  2.008790e+11  2.033801e+11  2.501081e+09  3.998272e-10  4.820544e-08


 TR Solving trust region problem, radius 2.000000e+00; starting on first irreducible block
 TR Coldstart. Seeking suitable initial Î»â, starting with 0
 TR Starting Newton iteration for Î»â with initial choice 0.000000e+00
 TR  iter        Î»            dÎ»       âhâ(Î»)â-radius
 TR      1  9.793842e+10  9.793842e+10  0.000000e+00


 iter inewton type    objectiv

As it can be seen, this case is faster than the one implemented above. Now it only takes 20 seconds to compute the beta and the result obtained is also very good. The results obtained are also very good, just the same as the previously ones calcualted. 


## d) Estimate the value of the regression coefficients of (1) by implementing the:

We are now going to estimate the values of beta implementing different methods. We will follow the formulas given in the notes of the subject and we will implement all these methods in python, withoud using any optimization libraries. Also, we will include the armijo rule in all of these methods to adjust the learning rate.

In the following chunk of codes you can see all these different methods implemented. As well as summary of some of the coefficients. Nevertheless, at the end of this point we will include a table summarizing all the results and obtaining conclusions


###     i. Gradient Method

This method is the simplest one and with less computational cost, as it does not evaluates the Hessian. In this case the descent direction is the negative of the gradient. For this method we spect good results but also high computation time.


In [295]:
(a,b) = X.shape

sigma = 0.1
alpha = 1
delta = 0.1
n_iter = 2000 # Maximum number of iterations
epsilon = 1e-5
tol_lsg = 10000
beta=0.2 #Set beta for the armijo rule

beta_lsg = np.zeros(b) # initial value for beta

OF_iter_lsg = np.zeros(n_iter)
tol_iter_lsg = np.zeros(n_iter)
alpha_iter = np.zeros(n_iter)

time_start_lsg = time.time()

i = 0
f_lsg=0
g_lsg=0
h_lsg=0

while (i <= n_iter-2) and (tol_lsg > epsilon):
    i = i + 1
    grad = ridreg_der(beta_lsg,X,Y,5) # Gradient vector
    g_lsg+=1
    ddirect =  -grad
    while(ridreg(beta_lsg-(alpha*grad),X,Y,5)>=ridreg(beta_lsg,X,Y,5)+sigma*alpha*np.dot(-grad.T,grad)):
        alpha=alpha*beta
        f_lsg+=2

    beta_lsg = beta_lsg + alpha*ddirect
    
    OF_iter_lsg[i] = ridreg(beta_lsg, X, Y,5)
    f_lsg+=1

    tol_lsg = np.linalg.norm(grad,ord=2)
    tol_iter_lsg[i] = tol_lsg
    alpha_iter[i] = alpha
    
time_elapsed_lsg = (time.time() - time_start_lsg)

i_lsg=i

print('Elapsed time = %8.5f' %(time_elapsed_lsg))
print('\nNumber of iterations = %5.0f' %i)

print('\nNumber of objective function evaluations = ',f_lsg)
print('\nNumber of gradient evaluations = ',g_lsg)

print('Objective function   = %11.5f' %OF_iter_lsg[i])
print('Optimality tolerance = %11.5f' %tol_lsg)

print('\nValues of the least squares coefficients - gradient method:')
print('beta %-9s %7.3f' %('intercept',beta_lsg[0]))
for ii in np.arange(1,b):
    print('beta %-9s %7.3f' %(X[ii,],beta_lsg[ii]))

beta_err_lsg = np.linalg.norm(np.transpose(beta_ls_exact)-beta_lsg,ord=2)/np.linalg.norm(beta_lsg,ord=2)
print('\nBeta coefficient error = %10.5f' %beta_err_lsg)

Elapsed time = 252.25826

Number of iterations =  1999

Number of objective function evaluations =  2027

Number of gradient evaluations =  1999
Objective function   = 681356884.55785
Optimality tolerance = 33752992.77598

Values of the least squares coefficients - gradient method:
beta intercept   0.514
beta [1.         4.18850497 4.07901155 ... 3.55935989 5.87931235 4.11944605]   2.422
beta [1.         4.17744932 5.75325813 ... 8.0044034  6.90993718 6.67571407]   2.966
beta [1.         1.25183904 9.73657322 ... 4.49424951 7.8225746  5.27591846]   3.117
beta [1.         9.66465579 2.5783558  ... 3.84155718 4.55725694 1.55848807]   2.374
beta [1.         0.17741963 3.50249338 ... 3.80062744 2.84224529 3.94900249]   2.246
beta [1.         5.21932338 9.14306975 ... 3.7462063  1.81637676 0.44365579]   2.500
beta [1.         7.36144334 6.3373537  ... 4.88102055 6.66247978 8.00266655]   2.761
beta [1.         3.51410008 5.0535762  ... 8.19411305 6.73456265 8.00473411]   2.913
beta [1.      

### ii. Newton Method

This methods converges very fast, thanks to the use of the hessian. Also, for this reason its computational cost is also higher. The implementation of this method is pretty much the same as the previous one. However, in this case the descent direction is the product of the hessian inverted and the gradient.

In [348]:
# Implementation of Newton's method
(a,b) = X.shape

## Parameters for the algorithm

alpha = 1
n_iter = 2000 # Maximum number of iterations
epsilon = 1e-5
tol_lsnm = 100000
sigma = 0.1
delta = 0.1
beta=0.5

## Initial values for the variables and data containers

beta_lsnm=np.zeros(b)

OF_iter_lsnm = np.zeros(n_iter)
tol_iter_lsnm = np.zeros(n_iter)
alpha_iter = np.zeros(n_iter)

# Implement Newton's method

time_start_lsnm = time.time()

i = 0

f_lsnm=0
g_lsnm=0
h_lsnm=0

while (i <= n_iter-2) and (tol_lsnm > epsilon):
    i = i + 1
    grad = ridreg_der(beta_lsnm,X,Y,5)
    g_lsnm+=1
    hess = ridreg_hess(beta_lsnm,X,Y,5)
    h_lsnm+=1
    ddirect = np.dot(-np.linalg.inv(hess),grad)
    while(ridreg(beta_lsnm+alpha*np.ndarray.flatten(ddirect),X,Y,5)>ridreg(beta_lsnm,X,Y,5)+sigma*alpha*np.dot(ddirect.T,grad).item()):
       alpha=alpha*beta
       f_lsnm+=2
    beta_lsnm = beta_lsnm + alpha*ddirect
    OF_iter_lsnm[i] = ridreg(beta_lsnm, X, Y,5)
    f_lsnm+=1
    tol_lsnm = np.linalg.norm(grad,ord=2)
    tol_iter_lsnm[i] = tol_lsnm
    alpha_iter[i] = alpha



time_elapsed_lsnm = (time.time() - time_start_lsnm)
i_lsnm=i


print('Elapsed time = %8.5f' %(time_elapsed_lsnm))
print('\nNumber of iterations = %5.0f' %i_lsnm)

print('\nNumber of objective function evaluations = ',f_lsnm)
print('\nNumber of Gradient evaluations = ',g_lsnm)
print('\nNumber of Hessian evaluations = ',h_lsnm)

print('Objective function   = %11.5f' %OF_iter_lsnm[i])


print('\nValues of the least squares coefficients - Newton method:')
print('beta %-9s %7.3f' %('intercept',beta_lsnm[0]))
for ii in np.arange(0,b):
    print('beta %-9s %7.3f' %(X[ii],beta_lsnm[ii]))

beta_err_lsnm = np.linalg.norm(np.transpose(beta_ls_exact)-beta_lsnm,ord=2)/np.linalg.norm(beta_lsnm,ord=2)
print('\nBeta coefficient error = %10.5f' %beta_err_lsnm)


Elapsed time =  2.92406

Number of iterations =     3

Number of objective function evaluations =  9

Number of Gradient evaluations =  3

Number of Hessian evaluations =  3
Objective function   = 96469.83265

Values of the least squares coefficients - Newton method:
beta intercept   2.328
beta [1.         9.81577917 0.84218216 ... 2.52505245 7.82327871 5.94822201]   2.328
beta [1.         4.18850497 4.07901155 ... 3.55935989 5.87931235 4.11944605]   2.001
beta [1.         4.17744932 5.75325813 ... 8.0044034  6.90993718 6.67571407]   4.000
beta [1.         1.25183904 9.73657322 ... 4.49424951 7.8225746  5.27591846]   4.998
beta [1.         9.66465579 2.5783558  ... 3.84155718 4.55725694 1.55848807]   1.999
beta [1.         0.17741963 3.50249338 ... 3.80062744 2.84224529 3.94900249]   1.000
beta [1.         5.21932338 9.14306975 ... 3.7462063  1.81637676 0.44365579]   2.001
beta [1.         7.36144334 6.3373537  ... 4.88102055 6.66247978 8.00266655]   3.001
beta [1.         3.51410008 5

### iii. Quasi-newton method

For computing this methods we do have to make a few changes compared to how we have been computing the preovious ones. In this case que need to first initialize $B_{0}$ with and aproximation to the hessian.

Then I will need to update these $B{k}$ with an update rule. There are many different rules, but in this case I decieded to use the BFGS method. It will also be necesary to use some other variables to simply the equations such as $s_{k}$ and $y_{k}$. Finally the descent direction in this case will be the inverse of $B_{k}$ multiplied by the gradient.

In [349]:
alpha = 1
n_iter = 2000 # Maximum number of iterations
epsilon = 1e-5
tol_qn = 10000
sigma = 0.1
delta = 0.1
beta=0.25

## Initial values for the variables and data containers

OF_iter_qn = np.zeros(n_iter)
tol_iter_qn = np.zeros(n_iter)
alpha_iter = np.zeros(n_iter)

# Implement Newton's method

time_start_qn = time.time()

i = 1

f_qn=0
g_qn=0
h_qn=0

bk = ridreg_hess(beta_ls0,X,Y,5)
h_qn+=1
beta_qn=np.zeros((n_iter,b))

while (i <= n_iter-2) and (tol_qn > epsilon):
    
    grad=ridreg_der(beta_qn[i-1,],X,Y,5)
    g_qn+=1
    ddirect=np.dot(-np.linalg.inv(bk),grad)

    #Armijo Rule
    while(ridreg(beta_qn[i-1,]+alpha*ddirect,X,Y,5) > ridreg(beta_qn[i-1,],X,Y,5) + sigma*alpha*np.dot(ddirect.T,grad)):
        alpha=alpha*beta
        f_qn+=2

    beta_qn[i,] = beta_qn[i-1,] + alpha*ddirect

    tol=np.linalg.norm(beta_qn[i-1,]-beta_qn[i-1,],ord=2)

    yk=ridreg_der(beta_qn[i,:],X,Y,5) - grad
    g_qn+=1
    sk=beta_qn[i]-beta_qn[i-1,]
    sk=sk[np.newaxis]
    yk=yk[np.newaxis]

    bk=bk - (np.dot(np.dot(sk,bk).T,np.dot(sk,bk)))/np.asscalar(np.dot(np.dot(sk,bk),sk.T)) + np.dot(yk,yk.T)/np.asscalar(np.dot(yk,sk.T))
    
    OF_iter_qn[i] = ridreg(beta_qn[i,:], X, Y,5)
    f_qn+=1
    tol_qn = np.linalg.norm(beta_qn[i,]-beta_qn[i-1,],ord=2)
    tol_iter_qn[i] = tol_qn
    alpha_iter[i] = alpha

    i=i+1

i_qn=i
time_elapsed_qn=time.time() - time_start_qn

print('Elapsed time = %8.5f' %(time_elapsed_qn))

print('\nNumber of objective function evaluations = ',f_qn)
print('\nNumber of Gradient evaluations = ',g_qn)
print('\nNumber of Hessian evaluations = ',h_qn)

print('\nNumber of iterations = %5.0f' %i)
print('Objective function   = %11.5f' %OF_iter_qn[i-1])

print('beta %-9s %7.3f' %('intercept',beta_qn[i-1,][0]))
for ii in np.arange(1,b):
    print('beta', beta_qn[i-1,][ii])

beta_err_qn = np.linalg.norm(np.transpose(beta_ls_exact)-beta_qn[i-1,],ord=2)/np.linalg.norm(beta_qn[i-1,],ord=2)
print('\nBeta coefficient error = %10.5f' %beta_err_qn)

/var/folders/k5/qpnm3ydn32l0mc6qzyk6129c0000gn/T/ipykernel_10911/4039531594.py:50: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  bk=bk - (np.dot(np.dot(sk,bk).T,np.dot(sk,bk)))/np.asscalar(np.dot(np.dot(sk,bk),sk.T)) + np.dot(yk,yk.T)/np.asscalar(np.dot(yk,sk.T))


Elapsed time =  1.67884

Number of objective function evaluations =  14

Number of Gradient evaluations =  4

Number of Hessian evaluations =  1

Number of iterations =     3
Objective function   = 96469.83265
beta intercept   2.328
beta 2.0008622571807373
beta 4.000445838928543
beta 4.998029116517071
beta 1.9990033834153498
beta 1.0004804789962611
beta 2.0006968376394387
beta 3.0009740435393195
beta 4.000673320983116
beta 1.0018757443660873
beta 4.997834435212979
beta -0.0010398926936089072
beta 1.0001036266314456
beta 3.9978147885821875
beta 5.001446731375012
beta 0.9999159382959992
beta 0.00042424982926794747
beta 0.9993864425449309
beta 0.9970527887989403
beta 2.9974309490462328
beta 2.001622101258828
beta 2.0006012128122896
beta 4.0016656676508005
beta 5.001563889637771
beta 0.9991757400879292
beta 3.9971338345781158
beta -0.0007464786466476993
beta 3.001764607469173
beta 3.9995925779611863
beta 0.0010425783046645546
beta 1.9979167356871994
beta 4.001828195690306
beta 2.9989780104

We are now going to summarize all the results obtained in a table to better analyze the differences between all the methods calculated.

In [350]:
resultsd={"Gradient Method":[i_lsg,f_lsg,g_lsg,h_lsg,time_elapsed_lsg,beta_err_lsg],
"Newton Method":[i_lsnm,f_lsnm,g_lsnm,h_lsnm,time_elapsed_lsnm,beta_err_lsnm],
"Quasi-Newton Method":[i_qn,f_qn,g_qn,h_qn,time_elapsed_qn,beta_err_qn]
}

results=pd.DataFrame(resultsd)
results=pd.DataFrame(results.T)
results.rename(columns = {0 : 'Number of iterations', 1:"Function Evaluations",2:"Gradient evaluations",3:"Hessian evaluations",4:"Time elapsed",5:"Beta error"}, inplace = True)
results

,Number of iterations,Function Evaluations,Gradient evaluations,Hessian evaluations,Time elapsed,Beta error
Gradient Method,1999.0,2027.0,1999.0,0.0,252.258258,0.495921
Newton Method,3.0,9.0,3.0,3.0,2.924061,0.004300
Quasi-Newton Method,3.0,14.0,4.0,1.0,1.678842,0.004300


As it can be seen, the gradient method is the only one that does not converges, it ends with the maximum number of iterations, which was expected as this method converges too slow. Also, the results obtained are not very good. It is the highest error that we have obtained so far. The other two meethods are both very good. They do only iterate 3 times each and the time taken is very low for both. The newton Method evaluated the hessian 3 times, which is 3 times more at what the Quasi-Newton did. From what we have been concluding trough this analysis, the hessian is the most expensive one in terms of computational costs. 

Finally, the errors obtained are both very low, for the Newton and Quasi-Newton.


# e) Estimate the value of the regression coefficients of (1) by implementing the:

### i. Coordinate descent method

The coordinated descen optimizes at each step k, only one component of w. Therefore, we will have to update the derivative function to minimize so we can update each component one by one.

Now, the computational cost should not be very high as it is just evaluating the gradient.

In [359]:
(a, b) = X.shape

def ridreg_der_coord(beta_ls,index,X,Y,p):
    pp=np.array(-2*np.dot((Y-np.dot(X,beta_ls)).T,X[:,index])) + 2*p*beta_ls[index]
    aa=np.zeros([b,1])
    aa[index]=pp
    return aa.T


niter = 5000
epsilon = 1e-9
i = 1
alpha = 1e-10
OF_iter_coor = np.zeros(niter)
tol_iter_coor = np.zeros(niter)
error_coor_iter = []
tol_coor = 10000

f_coor=0
g_coor=0
h_coor=0

beta_coor = np.zeros([niter,b])
time_start_coor = time.time()

while (i < niter) and (tol_coor > epsilon):
    k = np.random.randint(b)

    gradk = ridreg_der_coord(np.atleast_2d(beta_coor[i-1,]).T,k,X,Y,5)
    g_coor+=1
    beta_coor[i,] = beta_coor[i-1,] - alpha*np.ndarray.flatten(gradk)
    tol = np.linalg.norm(gradk, ord = 2)
    OF_iter_coor[i]  = ridreg(beta_coor[i,].T, X, Y,5)
    f_coor+=1
    tol_iter_coor[i] = tol_coor
    error_coor_iter.append(np.linalg.norm(np.transpose(beta_ls_exact) - beta_coor[i,].T, ord = 2)/np.linalg.norm(beta_ls_exact, ord = 2))
    i +=1
    
i_coor=i    
time_elapsed_coor = (time.time() - time_start_coor)
print('time elapsed =',time_elapsed_coor)
print('betas coord =',beta_coor[i-1,])
print('betas exact =', beta_ls_exact.T)
print('number iterations =',i)
print('tolerance=',tol_coor) 
print('error', error_coor_iter[i-2]) 

time elapsed = 244.2248501777649
betas coord = [0.32761579 1.77631578 2.42832645 ... 0.94999525 2.10135041 1.99363397]
betas exact = [[2.74671218 2.00075993 4.00035929 ... 1.00027018 2.00000827 4.9950782 ]]
number iterations = 5000
tolerance= 10000
error 0.6730587468268708


### ii. Stochastic Gradient

In this case we are computing at each step k one random sample of the dataset. therefore the computations should be much faster for each iteration. 

Now the movement for this case is follows this formula:

$w_{k+1}=w_{k}-\alpha*\nabla f_{ik}(w_{k})$

We can implement it in python with a similar scheme as the one implemented before.

In [375]:
(a, b) = X.shape
niter = 2000
epsilon = 1e-9
i = 0
alpha = 1e-6
OF_iter_sg = np.zeros(niter)
tol_iter_sg = np.zeros(niter)
error_sg = np.zeros(niter)
tol_sg = 10
beta_sg = np.zeros(b)

f_sg=0
g_sg=0
h_sg=0


time_start_sg = time.time()

while (i < niter) and (tol > epsilon):
    k=random.choice(range(a))
    beta_sg = beta_sg - 2*alpha*(np.dot(beta_sg,X[k])-Y[k])*X[k] + 2*5*beta_sg*alpha
    g_sg+=1
    OF_iter_sg[i]  = ridreg(beta_sg, X, Y,5)
    f_sg+=1
    if i>0:
        tol = np.abs((OF_iter_sg[i]-OF_iter_sg[i-1])/OF_iter_sg[i-1])
    tol_iter_sg[i] = tol
    error_sg[i] = np.linalg.norm(np.transpose(beta_ls_exact) - beta_sg, ord = 2)/np.linalg.norm(beta_ls_exact, ord = 2)
    i +=1
 
time_elapsed_sg = (time.time() - time_start_sg)
i_sg=i
 
print('time elapsed =',time_elapsed_sg)
print('betas stoch =',beta_sg)
print('betas exact =', beta_ls_exact.T)
print('number iterations =',i_sg)
print('tolerance=',tol) 
print('final error =', error_sg[i-1]) 

time elapsed = 51.15525984764099
betas stoch = [0.51474181 2.76655255 2.35359588 ... 2.56038882 2.90177049 2.5016408 ]
betas exact = [[2.74671218 2.00075993 4.00035929 ... 1.00027018 2.00000827 4.9950782 ]]
number iterations = 2000
tolerance= 0.002022466246680117
final error = 0.5391255410549421


### iii. Other three techniques

For this case we will implement three different techniques also seen in the second chapter of the subject

- Mini Batch gradient descent

This method has some similiarities with the previous one. But instead of considering one sample we are considering each step a sample $S_{k}$ of size b. Then the gradient is approximated as:

$w_{k}=w_{k}-\frac{\alpha_{k}}{|S_{k}|}\sum_{i \in S_k} \nabla f_{i}(w_{k})$

Therefore we implement it in python:

In [366]:
(a,b)=X.shape
beta_mb=np.zeros(b) #initial value for beta
alpha=1e-7 
n_iter=5000#maximim number iteration
OF_iter=np.zeros(n_iter)
tol_iter_mb=np.zeros(n_iter)
alpha_iter=np.zeros(n_iter)
error_mb=np.zeros(n_iter)
i=0
tol=1000000
epsilon=1e-9

#### Number of samples to take into consideration in each iteration
subsetsize = 10
#### Calculate one set of subsetsize random index to choose radomly some samples
 
subsets = np.random.choice([x for x in range(0,a)],n_iter*subsetsize)
subsets.resize(n_iter,subsetsize)

f_mb=0
g_mb=0
h_mb=0

time_start_mb = time.time()

while (i <= n_iter-2) and (tol>epsilon):

    vector=np.zeros(b)

    for j in range(subsetsize):
        vector=vector+ridreg_der(beta_mb,X[subsets[j],:],Y[subsets[j],:],5)
        g_mb+=1

    beta_mb = beta_mb - (alpha/(subsetsize))*vector

    OF_iter[i] = ridreg(beta_mb, X, Y,5)
    f_mb+=1
    if i>0:
        tol = np.abs((OF_iter[i]-OF_iter[i-1])/OF_iter[i-1])
    tol_iter_mb[i] = tol
    error_mb[i] = np.linalg.norm(np.transpose(beta_ls_exact)-beta_mb,ord=2)/np.linalg.norm(beta_mb,ord=2)
    i=i+1
    
time_elapsed_mb = (time.time() - time_start_mb) 
i_mb=i

print('time elapsed =',time_elapsed_mb)
print('iterations =',i)
print('Objective Function value =', OF_iter[i-1])
print('Betas =',beta_mb)
print('Tolerance=',tol)
print('error=',np.linalg.norm(np.transpose(beta_ls_exact)-beta_mb,ord=2)/np.linalg.norm(beta_mb,ord=2))



time elapsed = 135.66423082351685
iterations = 4999
Objective Function value = 1122160923.5937867
Betas = [0.50999257 2.4007449  2.77618336 ... 2.25462594 2.83367819 2.4456947 ]
Tolerance= 7.202853139232662e-06
error= 0.6272906754635008


- Second order Stochastic Quasi newton

This method have some similarities with the one that we already implemented, the quasi newton method, but it is quite different as the ones that we have been coding within this section.

First of all this is a scond order method, so we have to consider again the hessian matrix.

The movement rule will be now updated as:

$w_{k+1} = w_{k} - \alpha_{k}H_{k}\nabla F{wk}$

$H_{k}$ starts as an approximation to the inverse of the hessian, but then gets updated within a setting shown in the class notes.

We have implemented this method with the following code:

In [369]:
alpha = 1
n_iter = 2000 # Maximum number of iterations
epsilon = 1e-6
tol = 10000
beta=0.25

beta_sqn = np.zeros((n_iter,b)) # initial value for beta

## Initial values for the variables and data containers

OF_iter = np.zeros(n_iter)
tol_iter_sqn = np.zeros(n_iter)
alpha_iter = np.zeros(n_iter)
error_sqn=np.zeros(n_iter)

# Implement Newton's method

f_sqn=0
g_sqn=0
h_sqn=0

time_start_sqn = time.time()

i = 1
Hk=np.linalg.inv(ridreg_hess(beta_sqn[i-1,],X,Y,5))
h_sqn+=1

while (i <= n_iter-2) and (tol > epsilon):

    grad=ridreg_der(beta_sqn[i-1,],X,Y,5)
    g_sqn+=1
    alpha=beta/i
    beta_sqn[i,] = beta_sqn[i-1,] - alpha*np.dot(Hk,grad)
    sk=beta_sqn[i,] - beta_sqn[i-1,]
    sk=sk[np.newaxis]
    vk = ridreg_der(beta_sqn[i,],X,Y,5) + ridreg_der(beta_sqn[i-1,],X,Y,5)
    g_sqn+=2
    vk=vk[np.newaxis]

    Hk= np.dot(np.dot((np.identity(b)) - (np.dot(vk.T,sk)/np.asscalar(np.dot(vk,sk.T))).T,Hk),
    (np.identity(b)) - (np.dot(vk.T,sk)/np.asscalar(np.dot(vk,sk.T)))) + (np.dot(sk.T,sk)/np.dot(sk,vk.T))

    OF_iter[i] = ridreg(beta_sqn[i,], X, Y,5)
    f_sqn+=1
    tol = np.linalg.norm(grad,ord=2)
    tol_iter_sqn[i] = tol
    error_sqn[i] = np.linalg.norm(np.transpose(beta_ls_exact)-beta_sqn[i,],ord=2)/np.linalg.norm(beta_ls_exact,ord=2)

    i+=1

    
 
time_elapsed_sqn = (time.time() - time_start_sqn) 
i_sqn=i

print('time elapsed =',time_elapsed_sqn)
print('iterations =',i)
print('Objective Function value =', OF_iter[i])
print('beta %-9s %7.3f' %('intercept',beta_sqn[i-1,][0]))
for a,ii in enumerate(np.arange(1,b)):
    print('beta %3d %7.3f' %(a,beta_sqn[i-1,][ii]))

print('Tolerance=',tol)
print('error=',error_sqn[i-1,])
    



/var/folders/k5/qpnm3ydn32l0mc6qzyk6129c0000gn/T/ipykernel_10911/2712211047.py:40: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  Hk= np.dot(np.dot((np.identity(b)) - (np.dot(vk.T,sk)/np.asscalar(np.dot(vk,sk.T))).T,Hk),
/var/folders/k5/qpnm3ydn32l0mc6qzyk6129c0000gn/T/ipykernel_10911/2712211047.py:41: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  (np.identity(b)) - (np.dot(vk.T,sk)/np.asscalar(np.dot(vk,sk.T)))) + (np.dot(sk.T,sk)/np.dot(sk,vk.T))


time elapsed = 510.6245369911194
iterations = 1999
Objective Function value = 0.0
beta intercept   0.552
beta   0   0.475
beta   1   0.949
beta   2   1.185
beta   3   0.474
beta   4   0.237
beta   5   0.474
beta   6   0.712
beta   7   0.949
beta   8   0.238
beta   9   1.185
beta  10  -0.000
beta  11   0.237
beta  12   0.948
beta  13   1.186
beta  14   0.237
beta  15   0.000
beta  16   0.237
beta  17   0.236
beta  18   0.711
beta  19   0.475
beta  20   0.474
beta  21   0.949
beta  22   1.186
beta  23   0.237
beta  24   0.948
beta  25  -0.000
beta  26   0.712
beta  27   0.949
beta  28   0.000
beta  29   0.474
beta  30   0.949
beta  31   0.711
beta  32   1.186
beta  33   1.185
beta  34   1.187
beta  35   0.949
beta  36   0.712
beta  37   0.712
beta  38   0.474
beta  39   0.475
beta  40   0.949
beta  41   1.185
beta  42   0.000
beta  43   0.237
beta  44   0.949
beta  45   0.711
beta  46   0.237
beta  47   0.475
beta  48   1.185
beta  49   0.237
beta  50   0.475
beta  51   0.474
beta  52   

- Momentum 

From the "Other Methods" section we will implement the momentum method, which is very used in deep learning. Now we have to fix the parameters $\alpha$ and $\beta$ and implement the movement fucntion as follows:

$w_{k+1}=w_{k}-\alpha \nabla F(w_{k}) + \beta_{k}(w_{k}-w_{k-1})$

This method is specially different as we have to consider i-1,i and i+1 for each iteration. We have implemented it in the following chunk of code:

In [370]:
alpha = 1e-10
n_iter = 2000 # Maximum number of iterations
epsilon = 1e-6
tol = 10000
sigma = 0.1
delta = 0.1
beta=0.25

beta_om = np.zeros((n_iter,b)) # initial value for beta

OF_iter = np.zeros(n_iter)
tol_iter_om = np.zeros(n_iter)
alpha_iter = np.zeros(n_iter)
error_om=np.zeros(n_iter)

f_om=0
g_om=0
h_om=0

# Implement Newton's method
time_start_om = time.time()

i = 2

while (i <= n_iter-2) and (tol > epsilon):

    gradd=ridreg_der(beta_om[i-1,],X,Y,5)
    g_om+=1
    beta_om[i,]= beta_om[i-1,] - alpha*gradd + beta*(beta_om[i-1,] - beta_om[i-2,])

    tol = np.linalg.norm(ridreg_der(beta_om[i,],X,Y,5), ord = 2)
    g_om+=1
    OF_iter[i]  = ridreg(beta_om[i,], X, Y,5)
    f_om+=1
    tol_iter_om[i] = tol
    error_om[i]=np.linalg.norm(np.transpose(beta_ls_exact)-beta_om[i,],ord=2)/np.linalg.norm(beta_ls_exact,ord=2)
    i+=1

time_elapsed_om=time.time() - time_start_om

i_om=i

print('time elapsed =',time_elapsed_om)
print('iterations =',i)
print('Objective Function value =', OF_iter[i])
print('Betas =',beta_om[i-1,])
print('Tolerance=',tol)
print('error=',error_om[i-1])


time elapsed = 237.7290198802948
iterations = 1999
Objective Function value = 0.0
Betas = [0.51421069 2.44727335 2.90122768 ... 2.2591287  2.37910535 3.03793409]
Tolerance= 35558399.018772356
error= 0.4410871255624027


Once again, to summarize all the results we are going to generate a table so we can compare the models easily.

In [377]:
resultse={"Cordinate Descent":[i_coor-1,f_coor,g_coor,h_coor,time_elapsed_coor,error_coor_iter[i_coor-2]],
"Stochastic Gradient":[i_sg,f_sg,g_sg,h_sg,time_elapsed_sg,error_sg[i_sg-1]],
"Mini batch Gradient Descent":[i_mb,f_mb,g_mb,h_mb,time_elapsed_mb,error_mb[i_mb-1]],
"Second order Stochastic QuasiNewton":[i_sqn,f_sqn,g_sqn,h_sqn,time_elapsed_sqn,error_sqn[i_sqn-1]],
"Momentum Method":[i_om,f_om,g_om,h_sqn,time_elapsed_om,error_om[i_om-1]]
}

resultse=pd.DataFrame(resultse)
resulte=pd.DataFrame(resultse.T)
resulte.rename(columns = {0 : 'Number of iterations', 1:"Function Evaluations",2:"Gradient evaluations",3:"Hessian evaluations",4:"Time elapsed",5:"Beta error"}, inplace = True)
resulte


,Number of iterations,Function Evaluations,Gradient evaluations,Hessian evaluations,Time elapsed,Beta error
Cordinate Descent,4999.0,4999.0,4999.0,0.0,244.224850,0.673059
Stochastic Gradient,2000.0,2000.0,2000.0,0.0,51.155260,0.539126
Mini batch Gradient Descent,4999.0,4999.0,49990.0,0.0,135.664231,0.627291
Second order Stochastic QuasiNewton,1999.0,1998.0,5994.0,1.0,510.624537,0.762872
Momentum Method,1999.0,1997.0,3994.0,1.0,237.729020,0.441087


Before making any comparaissons, it will be neccesary to mention that to obtain better insights of the results, they should all have been computed under the same circunstances, i.e. having all of them the same number of maximum iterations. 

f) (2 points) Consider the constrained problem:

\begin{align*}
  \text{minimize}_\beta \quad & \|y-X\beta\|^2_2
\end{align*}

\begin{align*}
  \text{s.t.} \sum^{k}_{i=1} \beta_{i} \leq 100
\end{align*}

Estimate optimal value of the regression coefficients in (2) by implementing a barrier algorithm.

Following the barrier method our constrained problem can be turned into a unconstrained problem by the following form:

\begin{align*}
\text{minimize}_\beta \hspace{1cm} f(x)-\mu log(\sum_{i=1}^k c_{i}(x))
\end{align*}

In this case we only have one constraint. We can rewrite it as:

\begin{align*}
-\sum_{i=1}^K \beta_{i}+100 \geq 0
\end{align*}

\begin{align*}

\end{align*}

In [316]:
def f(beta,X,Y,mu):
    beta = np.matrix(beta)
    z = Y - np.dot(X,beta.T)
    zz=np.dot(z.T,z)
    cons=mu*np.log(-np.sum(beta[0,])+100)
    return(zz-cons)
    

def f_der(beta,X,Y,mu):
    beta = np.matrix(beta)
    pp = -2*np.dot((Y-np.dot(X,(beta).T)).T,X)
    aa = np.squeeze(np.asarray(pp))
    cons=(len(beta)*mu)/(-np.sum(beta)+100)
    return (aa-cons)

def f_hess(beta,X,Y,mu):
    beta_ls = np.matrix(beta)
    ss = 2*np.dot(np.transpose(X),X)
    k=len(beta_ls[0,])
    cons=(k*k*mu)/np.power((-np.sum(beta[0,:])+100),2)
    return ss-cons

mu=0.1

In [312]:
beta=np.zeros(b)
beta=np.matrix(beta)
print(np.sum(beta[0,]))

0.0


In [317]:
beta_ls0_p=np.zeros(nvars+1)

time_start_p = time.process_time()

res = minimize(f, beta_ls0_p, args=(X, Y,mu), method='Newton-CG', jac=f_der, hess=f_hess, options={'disp': True})

time_elapsed_p = (time.process_time() - time_start_p)

print('\nValues of the least squares coefficients obtained with Nelder-Mead:')
for i in range(nvars+1):
    print('beta %3d %7.3f' %(i,res.x[i]))

print('Elapsed time = %8.5f' %(time_elapsed_p))

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [302]:
np.sum(beta_ls0_p)
print(sum(res.x))

2573.462294454188
